In [2]:
### FIXME
# Parallelizing for each stock
###

import yfinance as yf
from scipy.optimize import least_squares, minimize
import numpy as np
import pandas as pd
from sklearn.metrics import r2_score
import pytz

In [95]:
delta_t = 2
dt = 1/252
max_delta = 1000

# PREPARE DATA
load_from = pd.to_datetime('2005-01-01').replace(tzinfo=pytz.UTC) 
train_start_date = pd.to_datetime('2010-01-01').replace(tzinfo=pytz.UTC)
test_start_date = pd.to_datetime('2020-01-01').replace(tzinfo=pytz.UTC)
test_end_date = pd.to_datetime('2023-01-01').replace(tzinfo=pytz.UTC)
data = yf.Ticker("GOOGL").history(start=load_from, end=test_end_date) 

# data['ret_lead'] = data.ret.shift(- delta_t)
data['ret_lead'] = (data['Close'].shift(-delta_t) - data['Open'].shift(-1)) / data['Open'].shift(-1)
data.loc[1:,'ret'] = np.diff(data['Close']) / data['Close'].iloc[1:]
data['time'] = np.arange(1, len(data)+1) * dt

# FIXME
x = data[['time', 'ret_lead', 'ret']].copy()

lags = np.arange(0, max_delta)
x = x.merge(pd.DataFrame({
        f'r_(t-{lag})': x.ret.shift(lag)
        for lag in lags
    }), left_index=True, right_index=True)

# Remove rows with missing values
x = x.dropna()

# SPLIT TRAIN/TEST
#idx_train = np.arange(0, round(len(x) * 0.9))
cols = [f'r_(t-{lag})' for lag in range(max_delta)]
x_train = x.loc[train_start_date:test_start_date, cols]
y_train = x.ret_lead.loc[train_start_date:test_start_date]
x_test = x.loc[test_start_date:test_end_date].loc[:, cols]
y_test = x.ret_lead.loc[test_start_date:test_end_date]
x_train.shape, y_train.shape, x_test.shape, y_test.shape

/tmp/ipykernel_31090/1299033885.py:14: FutureWarning: Slicing a positional slice with .loc is not supported, and will raise TypeError in a future version.  Use .loc with labels or .iloc with positions instead.
  data.loc[1:,'ret'] = np.diff(data['Close']) / data['Close'].iloc[1:]


((2516, 1000), (2516,), (754, 1000), (754,))

In [96]:
def exponential_kernel(lambda_x, returns):
    timestamps = np.arange(returns.shape[1]) * dt
    
    weights = lambda_x * np.exp(-lambda_x * timestamps)
    
    return np.sum(weights * returns, axis=1)


def d_exponential_kernel(lambda_x, returns):
    timestamps = np.arange(returns.shape[1]) * dt
    
    weights = np.exp(-lambda_x * timestamps)
    weights2 = lambda_x * np.exp(-lambda_x * timestamps)

    return np.sum(weights * returns, axis=1) + np.sum(timestamps * weights2 * returns, axis=1)

def residuals(params, returns, y, return_preds=False):
    beta0, beta1, beta2, lambda10, lambda11, theta1, lambda20, lambda21, theta2 = params

    # Calculate the exponential and moving average terms
    exp_term1 = np.array(
        (1 - theta1) * exponential_kernel(lambda10, returns) +
        theta1 * exponential_kernel(lambda11, returns)
    )

    exp_term2 = np.array(
        (1 - theta2) * exponential_kernel(lambda20, returns**2) +
        theta2 * exponential_kernel(lambda21, returns**2)
    )

    # Calculate the predicted values
    y_pred = beta0 + beta1 * exp_term1 + beta2 * np.sqrt(exp_term2)
    
    if return_preds:
        return y_pred
    
    # Calculate the least square error
    mse = y_pred - y
    return mse

def jacobian(params, returns, y):
    beta0, beta1, beta2, lambda_10, lambda_11, theta1, lambda_20, lambda_21, theta2 = params
    
    grad_beta0 = np.ones(len(y))
    
    grad_beta1 = np.array(
        (1 - theta1) * exponential_kernel(lambda_10, returns) +
        theta1 * exponential_kernel(lambda_11, returns)
    )
    
    grad_beta2 = np.array(
        np.sqrt((1 - theta2) * exponential_kernel(lambda_20, returns**2) +
        theta2 * exponential_kernel(lambda_21, returns **2)
    ))
    
    grad_theta1 = np.array(
         beta1 * (exponential_kernel(lambda_11, returns) - exponential_kernel(lambda_10, returns))
    )
    
    grad_theta2 = np.array(
        (beta2 / (2 * grad_beta2)) * (exponential_kernel(lambda_21, returns**2)
                                      - exponential_kernel(lambda_20, returns**2))                         
    )
    
    grad_lambda10 = np.array(
       beta1 * (1 - theta1) * (d_exponential_kernel(lambda_10, returns))
    )
    
    
    grad_lambda11 = np.array(
       beta1 * theta1 * d_exponential_kernel(lambda_11, returns)
    )
    
    grad_lambda20 = np.array(
        (beta2 / (2 * grad_beta2)) * ((1 - theta2) * (d_exponential_kernel(lambda_20, returns**2)))
    )

    grad_lambda21 = np.array(
        (beta2 / (2 * grad_beta2)) * (theta2 * d_exponential_kernel(lambda_21, returns**2))
    )

    gradient_output = np.vstack((grad_beta0, grad_beta1, grad_beta2, grad_lambda10, grad_lambda11,
                                 grad_theta1, grad_lambda20, grad_lambda21,  grad_theta2)
                            )
    
    return gradient_output.T

In [97]:
# Define the initial parameters (given in the paper)
initial_params = np.array([0, -0.1, 0.7, 60, 10, 0.2, 15, 5, 0.5])
#initial_params = np.zeros(9)
# Constraint
#constraints = ({'type': 'ineq', 'fun': constraint_func})

# Define the bounds
lower_bounds = np.array([-np.inf, -np.inf, -np.inf, 0, 0, 0, 0, 0, 0])
upper_bounds = np.array([np.inf, np.inf, np.inf, np.inf, np.inf, 1, np.inf, np.inf, 1])


# # Perform the optimization
result = least_squares(residuals, initial_params, bounds=(lower_bounds,upper_bounds), 
                       args=(x_train, y_train**2), method='trf', verbose=2,jac=jacobian)
# Check if the optimization was successful
if result.success:
    # Retrieve the optimized parameters
    optimized_params = result.x
    print("Optimization successful!")
    print("Optimized parameters:", optimized_params)
else:
    print("Optimization failed. Message:", result.message)

   Iteration     Total nfev        Cost      Cost reduction    Step norm     Optimality   
       0              1         5.0553e+01                                    3.80e+02    
       1              2         1.4258e-03      5.06e+01       7.06e-01       2.38e-02    
       2              3         1.4257e-03      2.62e-08       6.45e+00       4.33e-02    
       3              4         1.4252e-03      5.79e-07       1.29e+00       2.52e-02    
       4              5         1.4251e-03      4.28e-08       3.65e+00       2.83e-02    
       5              6         1.4250e-03      7.78e-08       5.56e-01       1.98e-02    
       6              8         1.4250e-03      5.13e-08       2.38e-01       1.09e-02    
       7             10         1.4250e-03      2.09e-08       6.46e-02       3.00e-03    
       8             12         1.4250e-03      7.90e-10       3.27e-02       1.52e-03    
       9             13         1.4250e-03      2.97e-10       8.18e-03       3.81e-04    

In [101]:
y_test2 = (y_test**2)
ref = x.ret_lead.shift(delta_t).loc[test_start_date:test_end_date] ** 2
print(pd.DataFrame({'optimized_params':optimized_params,'intial_params':initial_params}))
preds = residuals(optimized_params, x_test, y_test, return_preds=True)

test = pd.DataFrame({'preds':abs(preds), 'oracle':y_test2})
test.index = x.loc[test_start_date:test_end_date].index

print(test.quantile(np.linspace(0,1,11)))

test['oracle_sign'] = np.sign(y_test2 - ref)
test['pred_sign'] = np.sign(test.preds - ref)
print(test[['oracle_sign','pred_sign']].value_counts().reset_index())

r2_score(test.oracle,test.preds), np.corrcoef(test.oracle, test.preds)[0,1], np.mean(test.oracle_sign == test.pred_sign)

   optimized_params  intial_params
0          0.000121            0.0
1         -0.000035           -0.1
2          0.001066            0.7
3         58.393828           60.0
4         19.273605           10.0
5          0.999978            0.2
6         12.562159           15.0
7          4.210716            5.0
8          0.147683            0.5
        preds        oracle
0.0  0.000253  1.354355e-09
0.1  0.000308  4.724507e-06
0.2  0.000362  2.247468e-05
0.3  0.000383  6.158439e-05
0.4  0.000418  1.287403e-04
0.5  0.000449  2.175864e-04
0.6  0.000483  3.405406e-04
0.7  0.000520  5.562887e-04
0.8  0.000565  9.188735e-04
0.9  0.000616  1.815771e-03
1.0  0.001073  1.199630e-02
   oracle_sign  pred_sign    0
0          1.0        1.0  325
1         -1.0       -1.0  203
2         -1.0        1.0  174
3          1.0       -1.0   52


(0.020351960617042253, 0.25208957926713266, 0.7002652519893899)